<a href="https://colab.research.google.com/github/mamuncseru/analysis_on_epileptic_seizure_bonn_dataset/blob/main/Epileptic_Seizure_Prediction_using_EEG_bonn_data_with_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [ ]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
from torch.optim import Adam, SGD

import keras
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input, LSTM
from keras.layers import BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint


# from tensorflow import keras
from tensorflow.keras import layers

from time import time

start_time = time()


## Reading Data

In [ ]:
bigA = pd.read_csv("./data/dataset_A.csv").values
bigA = pd.DataFrame(bigA.transpose())
bigB = pd.read_csv("./data/dataset_B.csv").values
bigB = pd.DataFrame(bigB.transpose())
bigC = pd.read_csv("./data/dataset_C.csv").values
bigC = pd.DataFrame(bigC.transpose())
bigD = pd.read_csv("./data/dataset_D.csv").values
bigD = pd.DataFrame(bigD.transpose())
bigE = pd.read_csv("./data/dataset_E.csv").values
bigE = pd.DataFrame(bigE.transpose())

bigA['label'] = [0] * len(bigA)
bigB['label'] = [0] * len(bigB)
bigC['label'] = [1] * len(bigC)
bigD['label'] = [1] * len(bigD)
bigE['label'] = [1] * len(bigE)


bigA

## Classifying data categories

## Experiment 1: Comparing CNN and BiLSTM on Epilepsy versus Healthy Data Categories

In [ ]:
D1 = np.concatenate([bigA, bigD, bigE])

In [ ]:
D1.shape

### Creating training (80), validation(10) and test(10) data from category 1 and 3


In [ ]:
number_of_rows = D1.shape[0]

random_indices = np.random.choice(number_of_rows, size=int(number_of_rows*0.8), replace=False)

label_train = D1[random_indices, -1]
data_train = D1[random_indices, :-1]

D1_rest = np.delete(D1, random_indices, 0)

number_of_rows = D1_rest.shape[0]
random_indices = np.random.choice(number_of_rows, size=int(number_of_rows*0.5), replace=False)

label_val = D1_rest[random_indices, -1]
data_val = D1_rest[random_indices, :-1]

D1_rest_rest = np.delete(D1_rest, random_indices, 0)

label_test = D1_rest_rest[:, -1]
data_test = D1_rest_rest[:, :-1]

data_train = np.expand_dims(data_train, axis=2)
data_val = np.expand_dims(data_val, axis=2)
data_test = np.expand_dims(data_test, axis=2)

print("train: ", data_train.shape)
print("validation: ", data_val.shape)
print("test: ", data_test.shape)

### Define model evaluation function

In [ ]:
def evaluate_model(history, X_test, y_test, model):
  scores = model.evaluate((X_test), y_test, verbose=0)
  print("Accuracy: %.2f%%" % (scores[1]*100))

  print(history)
  fig1, ax_acc = plt.subplots()
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.title('Model - Accuracy')
  plt.legend(['Training', 'Validation'], loc='lower right')
  plt.show()

  fig2, ax_loss = plt.subplots()
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Model - Loss')
  plt.legend(['Training', 'Validation'], loc='upper right')
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.show()
  target_names = ['1', '2', '3']

  y_true = []
  for element in y_test:
    y_true.append(np.argmax(element))

  prediction_proba = model.predict(X_test)
  prediction = np.argmax(prediction_proba, axis=1)
  cnf_matrix = confusion_matrix(y_true, prediction)

## Defining Bidirectional LSTM model

In [ ]:
def network_LSTM(X_train, y_train):
  im_shape = (X_train.shape[1], 1)
  inputs_lstm = Input(shape=(im_shape), name='inputs_lstm')

  dense = Dense(units=32, activation='relu', name='dense')(inputs_lstm)
  lstm = layers.Bidirectional(LSTM(units=128, name='lstm'))(dense)
  dropout = Dropout(0.3)(lstm)
  batch_normalization = BatchNormalization(name='batch_normalization')(dropout)

  dense_1 = Dense(units=64, activation='relu', name='dsn_')(batch_normalization)
  dropout_2 = Dropout(0.3, name='drpt')(dense_1)
  batch_normalization_1 = BatchNormalization(name='batch_normalization_1')(dropout_2)
  main_output = Dense(units=2, activation='softmax')(batch_normalization_1)

  model = Model(inputs=inputs_lstm, outputs=main_output)
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model

  

In [ ]:
model2 = network_LSTM(data_train, label_train)
print(model2.summary())

In [ ]:
# Train BiLSTM on epileptic vs healthy data
save_path = 'checkpoint_2'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=save_path,
    save_weight_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
)

history2 = model2.fit(data_train, label_train, epochs=100, batch_size=32, validation_data=(data_val, label_val), callbacks=[model_checkpoint_callback])

end_time = time()

elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time} seconds")

In [ ]:
evaluate_model(history2, data_test, label_test, model2)

In [ ]:
model2.save("model.h5")
# tf.saved_model.save(model, "saved_model")